### LSTM

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-Indicators.csv', parse_dates=['Date'], index_col=['Date'])
data

,Close,sma,ema20,ema50,ema200,rsi,macd,macd_signal,Close_future,Price_Up
Date,,,,,,,,,,
2015-04-04,253.697006,266.058734,255.661524,258.734938,307.338692,47.508324,-4.193133,-3.906538,260.597992,1
2015-04-05,260.597992,265.654567,256.131663,258.807999,306.873610,52.432292,-3.252122,-3.775655,255.492004,0
2015-04-06,255.492004,264.962268,256.070744,258.677960,306.362350,48.785829,-2.885117,-3.597547,253.179993,0
2015-04-07,253.179993,264.256468,255.795434,258.462354,305.833173,47.185618,-2.749132,-3.427864,245.022003,0
2015-04-08,245.022003,262.770301,254.769393,257.935281,305.228086,41.956153,-3.262042,-3.394700,243.675995,0
...,...,...,...,...,...,...,...,...,...,...
2024-11-28,95652.468750,84858.846094,90477.864743,81038.222170,67603.340183,65.862719,5520.472967,6077.440370,97461.523438,1
2024-11-29,97461.523438,85696.245573,91142.975095,81682.273200,67900.436534,68.175702,5415.826234,5945.117543,96449.054688,0
2024-11-30,96449.054688,86570.707812,91648.316009,82261.362670,68184.502386,65.500818,5191.352498,5794.364534,97279.789062,1


In [23]:
features = ['Close', 'sma', 'ema20', 'ema50', 'ema200', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target].to_numpy()


In [24]:
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [33]:
import numpy as np

sequence_length = 30

X_sequences = []
y_sequences = []

for i in range(sequence_length, len(X_scaled)):
    X_sequences.append(X_scaled[i-sequence_length:i])
    y_sequences.append(y[i])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.01, shuffle=False
)

In [49]:
# Model 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(LSTM(units=100, return_sequences=False))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))


model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [52]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5303 - loss: 0.6923 - val_accuracy: 0.4722 - val_loss: 0.6950
Epoch 2/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5158 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6959
Epoch 3/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5238 - loss: 0.6897 - val_accuracy: 0.5278 - val_loss: 0.6921
Epoch 4/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5235 - loss: 0.6915 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5317 - loss: 0.6859 - val_accuracy: 0.4722 - val_loss: 0.6961
Epoch 6/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5265 - loss: 0.6901 - val_accuracy: 0.4722 - val_loss: 0.6947
Epoch 7/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.5317 - loss: 0.6907 - val_accuracy: 0.4722 - val_loss: 0.6945
Epoch 8/50
109/109 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5164 - loss: 0.6913 - val_accu

In [53]:
# 9. Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Precisión del modelo LSTM: {accuracy:.2f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5394 - loss: 0.6914
Precisión del modelo LSTM: 0.53


In [54]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.53      1.00      0.69        19

    accuracy                           0.53        36
   macro avg       0.26      0.50      0.35        36
weighted avg       0.28      0.53      0.36        36

[[ 0 17]
 [ 0 19]]


c:\Users\ivoto\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ivoto\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ivoto\miniconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
